In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#
# **************************************************************************************************************************************************
#	Libraries
# **************************************************************************************************************************************************
#
import datetime
import numpy as np
import os
import pandas as pd
from keras import applications
from keras import optimizers
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.preprocessing.image import image, ImageDataGenerator
np.random.seed(1337)
#
# **************************************************************************************************************************************************
#	Global Variables
# **************************************************************************************************************************************************
#

# location of the test image
imagePath = '/content/drive/My Drive/Colab Notebooks/test/Benign/3_7.jpg'

# saved filename of model weights
weights_filename = '/content/drive/My Drive/Colab Notebooks/weights.h5'

# saved filename of result csv
results_filename = '/content/drive/My Drive/Colab Notebooks/results.csv'

# dimensions of the architecture
img_width, img_height = 224, 224

# directories of dataset
test_data_dir = '/content/drive/My Drive/Colab Notebooks/test'

# number of channels for the architecture
channels = 3

# total number of classes
total_classes = 2

# learning rate
learning_rate = 1e-4

model = applications.ResNet50(include_top = False, weights = 'imagenet', classes = total_classes, input_shape=(img_width, img_height, 3))
flat = Flatten()(model.layers[-1].output)
output = Dense(total_classes, activation="sigmoid")(flat)
loaded_model = Model(inputs=model.inputs, outputs=output)
loaded_model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr = learning_rate), metrics=['acc'])

loaded_model.load_weights(weights_filename, by_name = True)

def test_image(path,output):
    test_image = image.load_img(imagePath, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = loaded_model.predict(test_image)

    confidence = ["{:.10f}".format(r) for r in result[0]]
    if output: print("Confidence: ", result)

    if (result[0][0] == result[0][1]):
        if output: print("Thyroid Type: Unknown. Unable to classify this image.")
        return "Unknown", confidence
    elif (np.argmax(result, axis = 1) == 0):
        if output: print("Thyroid Type: Benign")
        return "Benign", confidence
    else:
        if output: print("Thyroid Type: Malignant")
        return "Malignant", confidence

# predict one image
test_image(imagePath, True)

# predict all test images
images = []
predictions = []
confidences = []

print("Predicting all test images..")
start = datetime.datetime.now()

count = 0
for (dirpath, dirnames, filenames) in os.walk(test_data_dir):
    for file in filenames:
        result, confidence = test_image(os.path.join(dirpath, file), False)
        images.append(file)
        predictions.append(result)
        confidences.append(confidence)
        count += 1

print("Writing " + str(count) + " results to CSV..")
results = pd.DataFrame({"Filename": images,
                      "Prediction": predictions,
                      "Confidence": confidences})
results.to_csv(results_filename, index = False)

end = datetime.datetime.now()
elapsed = end - start
print("Prediction completed after " + str(elapsed) + ".")

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Confidence:  [[0. 1.]]
Thyroid Type: Malignant
Predicting all test images..
Writing 480 results to CSV..
Prediction completed after 0:02:26.885122.
